## Install  libraries

In [1]:
!pip install mediapipe -q
!pip install --upgrade diffusers[torch] -q
!pip install transformers -q
!pip install accelerate -q
!pip install git+https://github.com/huggingface/diffusers -q
!pip install -qU controlnet_aux -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 936.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

# Clone CodeFormer

In [5]:
# Clone CodeFormer and enter the CodeFormer folder
%cd /content/
!rm -rf CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd CodeFormer

# Set up the environment
# Install python dependencies
!pip install -r requirements.txt
# Install basicsr
!python basicsr/setup.py develop

# Download the pre-trained model
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer
%cd /content/

/content
Cloning into 'CodeFormer'...
remote: Enumerating objects: 594, done.
remote: Counting objects: 100% (594/594), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 594 (delta 287), reused 493 (delta 269), pack-reused 0
Receiving objects: 100% (594/594), 17.31 MiB | 33.95 MiB/s, done.
Resolving deltas: 100% (287/287), done.
/content/CodeFormer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.4 MB/s eta 0:00:00
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.

# Import libraries

In [56]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import math
import numpy as np
from PIL import Image
from numpy import float32
import matplotlib.pyplot as plt
import requests
import torch
import PIL
from diffusers import ControlNetModel, StableDiffusionControlNetInpaintPipeline, DPMSolverMultistepScheduler,UniPCMultistepScheduler,EulerDiscreteScheduler
from diffusers.utils import load_image
import torch
from google.colab import userdata
import io
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import os
import shutil
import logging
from pathlib import Path
from tqdm import tqdm

# Mount google drive

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
def face_restoration(image_in):
    %cd /content/CodeFormer/
    upload_folder = 'inputs/user_upload'
    if os.path.isdir(upload_folder):
        shutil.rmtree(upload_folder)
    os.mkdir(upload_folder)
    image_in.save(os.path.join(upload_folder,'image.png'))

    # We set w to 0.7 for the whole images
    # you can add '--bg_upsampler realesrgan' to enhance the background
    #@markdown *Codeformer_Fidelity (0 for better quality, 1 for better identity)*
    CODEFORMER_FEDILITY = 0.7 # @param {type:"slider", min:0, max:1, step:0.05}
    !python inference_codeformer.py -w $CODEFORMER_FEDILITY --input_path inputs/user_upload --bg_upsampler realesrgan
    %cd /content/
    img_out = Image.open(f'/content/CodeFormer/results/user_upload_{CODEFORMER_FEDILITY}/final_results/image.png')
    return img_out


# Create Required Directories and Upload Images

1.   List item
2.   List item



In [44]:
project_dir_path = '/content/drive/MyDrive/touch-up-the-hair'
minority_color_mask_dir_path = 'output_dir/minority_color_masks'
majority_color_mask_dir_path = 'output_dir/majority_color_masks'
touched_dir_path = 'output_dir/touched'
restored_dir_path = 'output_dir/touched_and_restored'
directory_list = ['models', 'input_dir', minority_color_mask_dir_path, majority_color_mask_dir_path, touched_dir_path, restored_dir_path]
for directory in directory_list:
    directory_path = os.path.join(project_dir_path, directory)
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

In [45]:
selfie_multiclass_model_path = os.path.join(project_dir_path, "models", "selfie_multiclass_256x256.tflite")
selfie_multiclass_model_url = "https://storage.googleapis.com/mediapipe-models/image_segmenter/selfie_multiclass_256x256/float32/latest/selfie_multiclass_256x256.tflite"
if(not os.path.exists(selfie_multiclass_model_path)):
  # Download the model
  urllib.request.urlretrieve(selfie_multiclass_model_url,selfie_multiclass_model_path)

In [46]:
#@markdown # **Generate Class**
class GenerateMask(object):

    def __init__(self, image_path):

        self._image_BGR = cv2.imread(image_path)
        if self._image_BGR is None:
            raise ValueError(f"Failed to load image from path: {image_path}")

        # Convert image to RGB (OpenCV uses BGR by default)
        self._image_RGB = cv2.cvtColor(self._image_BGR, cv2.COLOR_BGR2RGB)

        # Model path
        self._HAIR_SEGMENTER_MODEL_PATH = "/content/drive/MyDrive/touch-up-the-hair/models/selfie_multiclass_256x256.tflite"

        # Create the options for ImageSegmenter
        base_options = python.BaseOptions(model_asset_path = self._HAIR_SEGMENTER_MODEL_PATH)
        self._options = vision.ImageSegmenterOptions(base_options = base_options, output_category_mask = True)

        self.MASK_COLOR = (255, 255, 255)  # Define MASK_COLOR
        self.BG_COLOR = (0, 0, 0)  # Define BG_COLOR

    def get_image_BGR(self):
        """
            return image_BGR
        """
        return self._image_BGR

    def get_image_RGB(self):
        """
            return image_RGB
        """
        return self._image_RGB


    def get_mask(self,CLASS_INDEX = 1):
        """
            Get the hair mask using Mediapipe's ImageSegmenter.
        Returns:
            - np.array: Hair mask image.
        """

        # Convert the image to Mediapipe's format
        image_mediapipe = mp.Image(image_format = mp.ImageFormat.SRGB, data = self._image_BGR)

        # Retrieve the masks for the segmented image
        with vision.ImageSegmenter.create_from_options(self._options) as segmenter:
            segmentation_result = segmenter.segment(image_mediapipe)
            category_mask = segmentation_result.category_mask.numpy_view()

            mask_image = np.zeros(self._image_BGR.shape, dtype = np.uint8)
            bg_image = mask_image.copy()
            mask_image[:] = self.MASK_COLOR
            bg_image[:] = self.BG_COLOR

            # Generate hair mask
            condition = np.stack((category_mask,) * 3, axis = -1) == CLASS_INDEX
            mask = np.where(condition, mask_image, bg_image)

            return mask

    def get_hair_mask(self):
        return self.get_mask()

    def get_face_mask(self):
        return self.get_mask(CLASS_INDEX = 3)


    def get_gray_mask(self,mask_path = None):

        if mask_path is None:
            # If hair_mask_path is not available, calculate hair mask on the fly
            mask = self.get_hair_mask()
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        else:
            # Load the binary mask
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        return mask

    def get_minority_hair_mask(self, mask_path = None, num_cluster = 2, model = "KMeans", majority=False):
        """
        Generate a hair minority mask based on the input image and its corresponding mask.

        Parameters:
        - image_path (str): Path to the input image.
        - mask_path (str): Path to the binary mask where hair regions are white and background is black.
        - num_cluster (int): Number of clusters for KMeans clustering (default is 3).
        - model (str): Model to use for clustering (default is "KMeans").
        - majority (bool): Whether to use the majority cluster (default is False).
        Returns:
        - np.array: Hair minority mask image.
        """

        mask = self.get_gray_mask(mask_path = mask_path)

        #Perform K-means clustering solely on hair pixels,
        #creating two clusters instead of analyzing the entire image with three clusters.
        #This approach is chosen to avoid potential clustering of
        #dark black hair with the background due to similar color tones.
        #Subsequently, identify non-zero pixels within the mask, representing the masked area.
        mask_indices = np.where(mask > 0)

        # Get the pixels within the masked area
        pixels = self._image_RGB[mask_indices].reshape((-1, 3))

        # Perform KMeans clustering on the pixels
        if model == "KMeans":
          model = KMeans(n_clusters = num_cluster, n_init = 10)
        elif model == "GaussianMixture":
          model = GaussianMixture(n_components = num_cluster, covariance_type = 'full')
        else:
          raise ValueError("Invalid model name. Valid options are 'KMeans' and 'GaussianMixture'.")

        model.fit(pixels)

        # Get the labels for each pixel
        labels = model.predict(pixels)

        # Find the cluster with the fewest pixels
        if majority == False:
          min_cluster_label = np.bincount(labels).argmin()
        else:
          min_cluster_label = np.bincount(labels).argmax()

        # Create a binary mask where pixels belonging to the minimum cluster class are white
        cluster_mask = np.zeros_like(mask)
        cluster_mask[mask_indices] = 255 * (labels == min_cluster_label)

        rgb_img_hair_minority_mask = cv2.cvtColor(cluster_mask, cv2.COLOR_BGR2RGB)

        return rgb_img_hair_minority_mask


    def get_gray_minority_mask(self, minority_mask_path = None):
        if minority_mask_path is None:
            # If hair_mask_path is not available, calculate hair mask on the fly
            minority_mask = self.get_minority_hair_mask()
            minority_mask = cv2.cvtColor(minority_mask, cv2.COLOR_BGR2GRAY)
        else:
            # Load the binary mask
            minority_mask = cv2.imread(minority_mask_path, cv2.IMREAD_GRAYSCALE)
        return minority_mask


In [47]:
# #@markdown # **Stable Diffusion Controlnet Pipeline Class**
# class StableDiffusionControlnetPipeline(object):
#     def __init__(self):
#         self.MODEL_PATHS = {
#             "RealVisInpaint": "Uminosachi/realisticVisionV51_v51VAE-inpainting"
#         }
#         self.CONTROLNET_PATHS = {
#             "ControlNetInpaint": "lllyasviel/control_v11p_sd15_inpaint",
#             "ControlNetCanny": "lllyasviel/control_v11p_sd15_canny",
#         }

#         self.schedulers = {
#             "UniPCMultistepScheduler": UniPCMultistepScheduler,
#             "DPMSolverMultistepScheduler": DPMSolverMultistepScheduler,
#             "EulerDiscreteScheduler": EulerDiscreteScheduler
#         }

#         self.device = "cuda" if torch.cuda.is_available() else "cpu"

#         #Loading the ControlNet Inpaint and Canny Models
#         self.controlnet = [
#             ControlNetModel.from_pretrained(CONTROLNET_PATH, torch_dtype=torch.float16).to(self.device) for _, CONTROLNET_PATH in self.CONTROLNET_PATHS.items()
#         ]
#         controlnet_inpaint_scale = 0.5 # @param {type:"number"}
#         controlnet_canny_scale = 0.2 # @param {type:"number"}
#         self.controlnet_scales = [controlnet_inpaint_scale, controlnet_canny_scale]


#         assert len(self.controlnet) == len(self.controlnet_scales)

#         #Loading the Stabel Difussion RealisticVision ControlNet Inpaint Pipeline
#         pipe = StableDiffusionControlNetInpaintPipeline.from_pretrained(
#             self.MODEL_PATHS["RealVisInpaint"],
#             controlnet=self.controlnet,
#             safety_checker=None,
#             requires_safety_checker=False,
#             torch_dtype=torch.float16,
#         ).to(self.device)
#         pipe.scheduler = self.schedulers["UniPCMultistepScheduler"].from_config(pipe.scheduler.config)
#         self.pipe = pipe

#     def make_inpaint_condition(self, image, image_mask):
#         image = np.array(image.convert("RGB")).astype(np.float32) / 255.0
#         image_mask = np.array(image_mask.convert("L")).astype(np.float32) / 255.0
#         assert image.shape[0:1] == image_mask.shape[0:1], "image and image_mask must have the same image size"
#         image[image_mask > 0.5] = -1.0  # set as masked pixel
#         image = np.expand_dims(image, 0).transpose(0, 3, 1, 2)
#         image = torch.from_numpy(image)
#         return image

#     def make_canny_condition(self, image):
#         image = np.array(image)
#         image = cv2.Canny(image, 100, 200)
#         image = image[:, :, None]
#         image = np.concatenate([image, image, image], axis=2)
#         image = Image.fromarray(image)
#         return image

#     def roundUp(self, input, round):
#         return input + round - (input % round)

#     # Function to Edit the Hair Roots of the Img with Stable Diffusion and ControlNet
#     def stable_diffusion_controlnet(self, image_path, PROMPT, NEGATIVE_PROMPT, minority_mask_path = None):
#         pillow_img = Image.open(image_path)
#         cv2_img = np.asarray(pillow_img)
#         if minority_mask_path is None:
#           # If hair_mask_path is not available, calculate hair mask on the fly
#           mask_generator = GenerateMask(image_path)
#           HAIR_ROOT_MASK = mask_generator.get_minority_hair_mask()
#         else:
#           # Load the binary mask
#             HAIR_ROOT_MASK = cv2.imread(minority_mask_path)

#         HAIR_ROOT_MASK = cv2.cvtColor(HAIR_ROOT_MASK, cv2.COLOR_BGR2GRAY)

#         kernel = np.ones((12, 12), np.uint8)#expansion gives better results
#         HAIR_ROOT_MASK = cv2.dilate(HAIR_ROOT_MASK, kernel, iterations=1)

#         mask_image = cv2.cvtColor(HAIR_ROOT_MASK, cv2.COLOR_BGR2RGB)
#         mask_image = Image.fromarray(mask_image)

#         # Resize the image to be divisible by 8
#         height = self.roundUp(pillow_img.height, 8)
#         width = self.roundUp(pillow_img.width, 8)
#         SEED = 42 # @param {type:"integer"}
#         num_inference_steps = 50 # @param {type:"integer"},
#         strength = 1.0 # @param {type:"number"},
#         guidance_scale = 4.0 # @param {type:"number"}

#         generator = torch.Generator(device=self.device).manual_seed(SEED)

#         inpaint_condition = self.make_inpaint_condition(pillow_img, mask_image)
#         canny_condition = self.make_canny_condition(pillow_img)
#         control_images = [inpaint_condition, canny_condition]
#         touched_image = self.pipe(
#             prompt = PROMPT,
#             image = pillow_img,
#             mask_image = mask_image,
#             num_inference_steps = num_inference_steps,
#             generator = generator,
#             control_image=control_images,
#             controlnet_conditioning_scale = self.controlnet_scales,
#             negative_prompt = NEGATIVE_PROMPT,
#             strength = strength,
#             height = height,
#             width = width,
#             guidance_scale = guidance_scale,

#         ).images[0]


#         return HAIR_ROOT_MASK, touched_image

#     def loss(image, min_color_mask, maj_color_mask):
#       """
#       This function computes the Wasserstein distance (aka Earth Mover's Distance) between the probability distributions of (grayscale) pixel intentsities between the minority and majority color segmentation masks.

#       In theory, having the touched up region's pixel color distribution appear as similar as possible to that of the majority color region is desirable because
#       the touched up region will blend in better with the remainder of the hair. This improves the color while preserving more "naturalness".  Natural hair texture is often lost because it
#       appears that the hair texture in an image is numerically represented by particular patterns of minor variations in RGB values (the RGB values of which are approximated by grayscale pixel intensity)
#       and that these patters are vulnerable to being "smoothed" out of existence by the Stable Diffusion model.  Attempting to optimize this loss function is attempting to minimize the amount of
#       "smoothness" afflicting the touched up region while pushing the mean (and median) pixel color in the direction of the majority color region.


#       Parameters:
#       image (numpy array): The original image.
#       min_color_mask (numpy array): The (grayscale) minority color segmentation mask.
#       maj_color_mask (numpy array): The (grayscale) majority color segmentation mask.

#       Returns:
#       loss (float): The Wasserstein distance between the probability distributions of (grayscale) pixel intentsities between the minority and majority color segmentation masks.
#       """

#       # Create black background image equal in size to the original image
#       colored_min_region = np.zeros_like(image)
#       colored_maj_region = np.zeros_like(image)

#       # Extract the indices of the pixel regions of each mask
#       min_mask_regions = np.where(min_color_mask > 0)
#       maj_mask_regions = np.where(maj_color_mask > 0)

#       # Map the colored pixels from the minority and majority color segmentation masks onto the black background image
#       colored_min_region[min_mask_regions] = image[min_mask_regions]
#       colored_maj_region[maj_mask_regions] = image[maj_mask_regions]

#       """
#       The two lines below convert RGB images to grayscale images.
#       NOTE: The formula to do, based on relative perception of color brightness, is the following: 0.299 ∙ Red + 0.587 ∙ Green + 0.114 ∙ Blue
#       The corresponding grayscale pixel value is a weighted sum of the RGB pixel values and represents the pixel "intensity".
#       """
#       gray_min_region = cv2.cvtColor(colored_min_region, cv2.COLOR_BGR2GRAY)
#       gray_maj_region = cv2.cvtColor(colored_maj_region, cv2.COLOR_BGR2GRAY)

#       # Flatten each greyscale image to a vector
#       gray_min_vector = gray_min_region.flatten()
#       gray_maj_vector = gray_maj_region.flatten()

#       # Extract nonzero pixel values (ie discard the black background pixels)
#       gray_min_pixels = gray_min_vector[gray_min_vector > 0]
#       gray_maj_pixels = gray_maj_vector[gray_maj_vector > 0]

#       # Compute histogram of pixel intensities for the minority color segmentation mask
#       min_mask_distribution = np.bincount(gray_min_pixels) / gray_min_pixels.size

#       # Compute histogram of pixel intensities for the majority color segmentation mask
#       maj_mask_distribution = np.bincount(gray_maj_pixels) / gray_maj_pixels.size

#       # Ensure that the probability distribtion sums to 1
#       assert(min_mask_distribution.sum() == 1)
#       assert(maj_mask_distribution.sum() == 1)

#       # Compute Wasserstein distance between the two histograms of pixel intensities
#       distribution_distance = scipy.stats.wasserstein_distance(min_mask_distribution, maj_mask_distribution, u_weights=None, v_weights=None)
#       loss = distribution_distance

#       return loss



In [48]:
class StableDiffusionControlnetPipeline(object):
    def __init__(self):
        self.MODEL_PATHS = {
            "RealVisInpaint": "Uminosachi/realisticVisionV51_v51VAE-inpainting"
        }
        self.CONTROLNET_PATHS = {
            "ControlNetInpaint": "lllyasviel/control_v11p_sd15_inpaint",
            "ControlNetCanny": "lllyasviel/control_v11p_sd15_canny",
        }

        self.schedulers = {
            "UniPCMultistepScheduler": UniPCMultistepScheduler,
            "DPMSolverMultistepScheduler": DPMSolverMultistepScheduler,
            "EulerDiscreteScheduler": EulerDiscreteScheduler
        }

        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        #Loading the ControlNet Inpaint and Canny Models
        self.controlnet = [
            ControlNetModel.from_pretrained(CONTROLNET_PATH, torch_dtype=torch.float16).to(self.device) for _, CONTROLNET_PATH in self.CONTROLNET_PATHS.items()
        ]

        #assert len(self.controlnet) == len(self.controlnet_scales)

        #Loading the Stable Diffusion RealisticVision ControlNet Inpaint Pipeline
        pipe = StableDiffusionControlNetInpaintPipeline.from_pretrained(
            self.MODEL_PATHS["RealVisInpaint"],
            controlnet=self.controlnet,
            safety_checker=None,
            requires_safety_checker=False,
            torch_dtype=torch.float16,
        ).to(self.device)
        pipe.scheduler = self.schedulers["UniPCMultistepScheduler"].from_config(pipe.scheduler.config)
        self.pipe = pipe


    def make_inpaint_condition(self, image, image_mask):
        image = np.array(image.convert("RGB")).astype(np.float32) / 255.0
        image_mask = np.array(image_mask.convert("L")).astype(np.float32) / 255.0
        assert image.shape[0:1] == image_mask.shape[0:1], "image and image_mask must have the same image size"
        image[image_mask > 0.5] = -1.0  # set as masked pixel
        image = np.expand_dims(image, 0).transpose(0, 3, 1, 2)
        image = torch.from_numpy(image)
        return image

    def make_canny_condition(self, image):
        image = np.array(image)
        image = cv2.Canny(image, 100, 200)
        image = image[:, :, None]
        image = np.concatenate([image, image, image], axis=2)
        image = Image.fromarray(image)
        return image

    def roundUp(self, input, round):
        return input + round - (input % round)

    # Function to dilate the mask (used in production only, NOT for generating minority and majority mask inputs to the loss function!!!)
    def dilate_mask(self, hair_root_mask):
        kernel = np.ones((12, 12), np.uint8)#expansion gives better results
        HAIR_ROOT_MASK = cv2.dilate(HAIR_ROOT_MASK, kernel, iterations=1)

        return HAIR_ROOT_MASK

    def get_mask(self, image_path, minority_mask_path = None):
        pillow_img = Image.open(image_path)
        cv2_img = np.asarray(pillow_img)
        if minority_mask_path is None:
          # If hair_mask_path is not available, calculate hair mask on the fly
          mask_generator = GenerateMask(image_path)
          HAIR_ROOT_MASK = mask_generator.get_minority_hair_mask()
        else:
          # Load the binary mask
            HAIR_ROOT_MASK = cv2.imread(minority_mask_path)

        HAIR_ROOT_MASK = cv2.cvtColor(HAIR_ROOT_MASK, cv2.COLOR_BGR2GRAY)

        mask_image = cv2.cvtColor(HAIR_ROOT_MASK, cv2.COLOR_BGR2RGB)
        mask_image = Image.fromarray(mask_image)

        return mask_image

    # Function to Edit the Hair Roots of the Img with Stable Diffusion and ControlNet
    def stable_diffusion_controlnet(self, image_path, mask_image, PROMPT, NEGATIVE_PROMPT, dilate=False, num_inference_steps=50, strength=1.0, guidance_scale=4.0, controlnet_inpaint_scale=0.5, controlnet_canny_scale=0.2):
        pillow_img = Image.open(image_path)

        """
        Dilate the (minority color) mask if the condition is True; this condition is 'False' by default
        because we do not want to dilate the mask before passing it to the loss function (when we are tuning parameters).
        However, we do want to dilate the mask when producing a touched up image for the production pipeline
        (after a satisfactory set of parameters has been chosed).
        """

        if dilate == True:
          mask_image = self.dilate_mask(mask_image)

        # Resize the image to be divisible by 8
        height = self.roundUp(pillow_img.height, 8)
        width = self.roundUp(pillow_img.width, 8)

        SEED = 42
        generator = torch.Generator(device=self.device).manual_seed(SEED)

        inpaint_condition = self.make_inpaint_condition(pillow_img, mask_image)
        canny_condition = self.make_canny_condition(pillow_img)
        control_images = [inpaint_condition, canny_condition]
        touched_image = self.pipe(
            prompt = PROMPT,
            image = pillow_img,
            mask_image = mask_image,
            num_inference_steps = num_inference_steps,
            generator = generator,
            control_image=control_images,
            controlnet_conditioning_scale = self.controlnet_scales,
            negative_prompt = NEGATIVE_PROMPT,
            strength = strength,
            height = height,
            width = width,
            guidance_scale = guidance_scale,

        ).images[0]

        return touched_image

    def loss(image, min_color_mask, maj_color_mask):
      """
      This function computes the Wasserstein distance (aka Earth Mover's Distance) between the probability distributions of (grayscale) pixel intentsities between the minority and majority color segmentation masks.

      In theory, having the touched up region's pixel color distribution appear as similar as possible to that of the majority color region is desirable because
      the touched up region will blend in better with the remainder of the hair. This improves the color while preserving more "naturalness".  Natural hair texture is often lost because it
      appears that the hair texture in an image is numerically represented by particular patterns of minor variations in RGB values (the RGB values of which are approximated by grayscale pixel intensity)
      and that these patters are vulnerable to being "smoothed" out of existence by the Stable Diffusion model.  Attempting to optimize this loss function is attempting to minimize the amount of
      "smoothness" afflicting the touched up region while pushing the mean (and median) pixel color in the direction of the majority color region.


      Parameters:
      image (numpy array): The original image.
      min_color_mask (numpy array): The (grayscale) minority color segmentation mask.
      maj_color_mask (numpy array): The (grayscale) majority color segmentation mask.

      Returns:
      loss (float): The Wasserstein distance between the probability distributions of (grayscale) pixel intentsities between the minority and majority color segmentation masks.
      """

      # Create black background image equal in size to the original image
      colored_min_region = np.zeros_like(image)
      colored_maj_region = np.zeros_like(image)

      # Extract the indices of the pixel regions of each mask
      min_mask_regions = np.where(min_color_mask > 0)
      maj_mask_regions = np.where(maj_color_mask > 0)

      # Map the colored pixels from the minority and majority color segmentation masks onto the black background image
      colored_min_region[min_mask_regions] = image[min_mask_regions]
      colored_maj_region[maj_mask_regions] = image[maj_mask_regions]

      """
      The two lines below convert RGB images to grayscale images.
      NOTE: The formula to do, based on relative perception of color brightness, is the following: 0.299 ∙ Red + 0.587 ∙ Green + 0.114 ∙ Blue
      The corresponding grayscale pixel value is a weighted sum of the RGB pixel values and represents the pixel "intensity".
      """
      gray_min_region = cv2.cvtColor(colored_min_region, cv2.COLOR_BGR2GRAY)
      gray_maj_region = cv2.cvtColor(colored_maj_region, cv2.COLOR_BGR2GRAY)

      # Flatten each greyscale image to a vector
      gray_min_vector = gray_min_region.flatten()
      gray_maj_vector = gray_maj_region.flatten()

      # Extract nonzero pixel values (ie discard the black background pixels)
      gray_min_pixels = gray_min_vector[gray_min_vector > 0]
      gray_maj_pixels = gray_maj_vector[gray_maj_vector > 0]

      # Compute histogram of pixel intensities for the minority color segmentation mask
      min_mask_distribution = np.bincount(gray_min_pixels) / gray_min_pixels.size

      # Compute histogram of pixel intensities for the majority color segmentation mask
      maj_mask_distribution = np.bincount(gray_maj_pixels) / gray_maj_pixels.size

      # Ensure that the probability distribtion sums to 1
      assert(min_mask_distribution.sum() == 1)
      assert(maj_mask_distribution.sum() == 1)

      # Compute Wasserstein distance between the two histograms of pixel intensities
      distribution_distance = scipy.stats.wasserstein_distance(min_mask_distribution, maj_mask_distribution, u_weights=None, v_weights=None)
      loss = distribution_distance

      return loss



# Instantiate the pipeline

In [49]:
SB_ControlNet_pipeline = StableDiffusionControlnetPipeline()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

# **Prompt**

In [51]:
def merge_prompts(prompts):
  return ", ".join(prompts)

In [52]:
# @markdown #**Positive Prompt**

PROMPT_1 = "(change the color of the hair roots to be like the rest of the hair color:1.2)" # @param {type:"string"}
PROMPT_2 = "raw photo, high detail, best quality, keep the same style of the hair" # @param {type:"string"}
PROMPT_3 = "beautiful hair, beautiful hair roots" # @param {type:"string"}
prompt_list = [PROMPT_1, PROMPT_2, PROMPT_3]
PROMPT = merge_prompts(prompt_list)
# @markdown #**Negative Prompt**

NEG_PROMPT_1 =  "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime)" # @param {type:"string"}
NEG_PROMPT_2 = "text, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers" # @param {type:"string"}
NEG_PROMPT_3 = "mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions" # @param {type:"string"}
NEG_PROMPT_4 = "extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs"# @param {type:"string"}
NEG_PROMPT_5 = "fused fingers, too many fingers, long neck" # @param {type:"string"}
negative_prompt_list = [NEG_PROMPT_1, NEG_PROMPT_2, NEG_PROMPT_3, NEG_PROMPT_4, NEG_PROMPT_5]
NEGATIVE_PROMPT = merge_prompts(negative_prompt_list)


# **Touch up the hair for a given list of images**

In [57]:
# Generate minority and majority color hair masks
demo_img_list = [1, 19, 21, 22, 32]
if __name__ == "__main__":
  for i in tqdm(demo_img_list):
    # Get image path
    IMG_PATH = os.path.join(project_dir_path,"input_dir/Images", "pic" + str(i) + ".png")

    # Generate masks
    minority_hair_mask = GenerateMask(IMG_PATH).get_minority_hair_mask()
    majority_hair_mask = GenerateMask(IMG_PATH).get_minority_hair_mask(majority=True)

    # Create minority and majority hair mask paths
    MINORITY_HAIR_MASK_PATH = os.path.join(project_dir_path, minority_color_mask_dir_path,"pic" + str(i) + "minority_color_mask.png")
    MAJORITY_HAIR_MASK_PATH = os.path.join(project_dir_path, majority_color_mask_dir_path,"pic" + str(i) + "majority_color_mask.png")

    # Save the results
    cv2.imwrite(MINORITY_HAIR_MASK_PATH, minority_hair_mask)
    cv2.imwrite(MAJORITY_HAIR_MASK_PATH, majority_hair_mask)

100%|██████████| 5/5 [00:07<00:00,  1.48s/it]


In [ ]:
def fit_pipeline(img_path):
  return 0

In [ ]:
# NOTE: You must upload image (or folder of images, in this case) to the 'input_dir' before running this function!

#demo_img_list = [1,19,21,22,32]
demo_img_list = [1]
if __name__ == "__main__":
  for i in tqdm(demo_img_list):
    # In this case, images are stored in an 'Images' folder placed in the 'input_dir' with names like 'pic1.png', 'pic2.png', etc.
    IMG_PATH = os.path.join(project_dir_path,"input_dir/Images", "pic" + str(i) + ".png")
    MINORITY_HAIR_MASK_PATH = os.path.join(project_dir_path, minority_color_mask_dir_path,"pic" + str(i) + "minority_color_mask.png")
    MAJORITY_HAIR_MASK_PATH = os.path.join(project_dir_path, majority_color_mask_dir_path,"pic" + str(i) + "majority_color_mask.png")

    # Process the image with the pipeline
    touched_image = stable_diffusion_controlnet(self, image_path, mask_image, PROMPT, NEGATIVE_PROMPT, dilate=False, num_inference_steps=50, strength=1.0, guidance_scale=4.0, controlnet_inpaint_scale=0.5, controlnet_canny_scale=0.2):
    touched_image_restored = face_restoration(touched_image) # Restore face as seperate function??!

    # Save the results
    TOUCHED_IMAGE_PATH = os.path.join(project_dir_path, touched_dir_path, "pic" + str(i) + "_touched.png")
    RESTORED_IMAGE_PATH = os.path.join(project_dir_path, restored_dir_path, "pic" + str(i) + "_restored.png")

    cv2.imwrite(MINORITY_HAIR_MASK_PATH, minority_hair_mask)
    touched_image.save(TOUCHED_IMAGE_PATH)
    touched_image_restored.save(RESTORED_IMAGE_PATH)

In [ ]:
# def apply_face_mask_overlay(image, face_mask, color = (0, 255, 255), alpha = 1.0):
#     pil_image = image.convert('RGB')
#     open_cv_image = np.array(pil_image)
#     # Convert RGB to BGR
#     input_image = open_cv_image[:, :, ::-1].copy()
#     # Load the input image and mask
#     gray_mask = cv2.cvtColor(face_mask, cv2.COLOR_BGR2GRAY)
#     mask_image = cv2.resize(gray_mask, input_image.shape[:2])

#     # Create a blank image with the same dimensions as the input image
#     overlay = np.zeros_like(input_image)

#     # Apply the overlay color to the blank image using the mask
#     overlay[mask_image > 0] = color

#     result = input_image.copy()
#     # Combine the input image and the overlay using alpha blending
#     result[mask_image > 0] = cv2.addWeighted(input_image[mask_image > 0], 0.1, overlay[mask_image > 0], alpha, 0)


#     # Convert the resulting image from OpenCV to PIL format
#     result_pil = Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

#     return result_pil

In [ ]:
  # demo_img_list = [1,19,21,22,32]
  # ReSize = (512,512)
  # img_titles = ["Original Image", "Touched Image", "Restored Image"]
  # if __name__ == "__main__":
  #   for n, i in enumerate(demo_img_list):
  #     fig, axes = plt.subplots(1, 3, figsize = (16,9))
  #     IMG_PATH =  os.path.join(project_dir_path,"input_dir", "pic" + str(i) + ".png")
  #     MINORITY_HAIR_MASK_PATH = os.path.join(project_dir_path, mask_dir_path,"pic" + str(i) + "_mask.png")
  #     TOUCHED_IMAGE_PATH = os.path.join(project_dir_path, touched_dir_path, "pic" + str(i) + "_touched.png")
  #     RESTORED_IMAGE_PATH = os.path.join(project_dir_path, restored_dir_path, "pic" + str(i) + "_restored.png")
  #     mask_generater = GenerateMask(IMG_PATH)
  #     face_mask = mask_generater.get_face_mask()
  #     image = Image.open(IMG_PATH)
  #     touched_image = Image.open(TOUCHED_IMAGE_PATH)
  #     restored_image = Image.open(RESTORED_IMAGE_PATH)
  #     img_list = [image.resize(ReSize), touched_image.resize(ReSize), restored_image.resize(ReSize)]
  #     for j, img_ in enumerate(img_list):
  #         img = apply_face_mask_overlay(img_,face_mask)

  #         axes[j].imshow(img)
  #         axes[j].axis('off')
  #         if n == 0:
  #           axes[j].set_title(img_titles[j])

  #     plt.show()


